# forage stats
## Methods to read in and analyze the raw data produced by the foraging game

In [12]:
import math

def getDist( x2, y2, x1, y1):
    x = (x2-x1) * (x2-x1)
    y = (y2-y1) * (y2-y1)
    d = math.sqrt(x + y)
    return d

#outside, outside, inside
def getAngle( x3, y3, x2, y2, x1, y1):
    #first find the length of the segments
    a = getDist(x3, y3, x2, y2)
    b = getDist(x2, y2, x1, y1)
    c = getDist(x3, y3, x1, y1)
    if a == 0 or b == 0:
        print 'Got a 0 angle'
        return -1
    
    #print '{0} {1} {2}'.format(a, b, c)
    #a^2 + b^2 - c^2 / 2ab
    inside = ( (a*a) + (b*b) - (c*c) ) / ( 2 * a * b ) #named because its the math INSIDE the acos
    if inside > 1.0:
        inside = 1.0
    elif inside < -1.0:
        inside = -1.0
    angle = math.acos( inside )
    angle = math.degrees(angle)
    #if angle == None:
    assert angle <= 180
    return angle

def mean_func( numbers):
    sum = 0.0
    for number in numbers:
        sum += number
    #print "Sum:{0} len:{1}".format(sum, len(numbers))
    aver = sum / len(numbers)
    return aver

def std( numbers):
    deviation = 0.0
    aver = mean_func(numbers)
    for number in numbers:
        deviation += (number - aver) * (number - aver)
    deviation = deviation / len(numbers)
    deviation = math.sqrt(deviation)
    return deviation

def get_bounds( x, y):
    width = 1280 / 15 / 2
    height = 1024 / 15 / 2
    x_left = int(x - width)
    if x_left < 0:
        x_left = 0
    x_right = int(x + width)
    if x_right >= 1280:
        x_right = 1280 - 1
    y_up = int(y - height)
    if y_up < 0:
        y_up = 0
    y_down = int(y + height)
    if y_down >= 1024:
        y_down = 1024-1
    return (x_left, x_right, y_up, y_down)
    
def allenConvert( list):
    width = 1280
    height = 1024
    #[height][width] blech
    matrix = [[0 for x in xrange(width)] for x in xrange(height)]  
    for k in range(len(list)):
        #print list[k]
        x = list[k][0]
        y = list[k][1]
        if x >= 1280:
            x = 1280 - 1
        if x < 0:
            x = 0
        if y >= 1024:
            y = 1024 - 1
        if y < 0:
            y = 0
        matrix[y][x] = matrix[y][x] + 1
    return matrix

In [2]:
import numpy as np

def get_stats( name, out_name, trial_num, file):
    #check if the directory already exists, if it doesn't make it
    if not os.path.exists('datas'):
        os.mkdir('datas')
    #declare a few file objects
    out_file = open('datas\\{0}-{1}-{2}-clicks.txt'.format(out_name, name, trial_num), 'w') #zoomed out
    angle_file = open('datas\\{0}-{1}-{2}-angle.txt'.format(out_name, name, trial_num), 'w') #angles!
    outfile = open('datas\\data-{0}.txt'.format(out_name), 'a') #summary stats
    
    #variables
    space = [ [0 for col in range(1280)] for row in range(1024) ]
    subj_id = name[0:7]
    trial_id = 0
    clustering = 0 #0 is random, 1 is clustered [-8]
    num_stars = 0
    score = 0
    totalScore = 0
    path_lengths = []
    in_lengths = []
    angles = []
    thresh_angles = []
    zoom_lengths = []
    zoom_list = []
    skip_lengths = []
    number_of_dives = 0
    
    #start the ship in the center
    past_shipX2 = 640.0 
    past_shipY2 = 512.0
    past_shipX = 640.0 
    past_shipY = 512.0

    current_shipX = 640.0
    current_shipY = 512.0
    fixations = []
    collections = []
    outX = 0.0
    outY = 0.0
    zX = 0.0
    zY = 0.0
    zX2 = 640.0
    zY2 = 512.0
    zoomed = 0

    
    angle_thresh = 10
    angle_segments = [] #temp list for adding segments together
    angle_paths = [] #hold the full pahs determined by angle changes
    
    landed = 1 #variable to keep track of whether the probe has landed
    file = file.split('\n')
    file_length = len(file)
    for iter, line in enumerate(file):
        if len(line) > 0:
            #print line
           
            if line[:5] == "maps/":
                num_stars = int(line[5:9])
                clustering = int(line[14])
                
                
            line = line.split()
            #if an interesting event occurs...
            if line[0] == 'Click':

                current_shipX = float(line[2])
                current_shipY = float(line[3])
                if current_shipX <= 0 or current_shipY <= 0 or current_shipX > 1280 or current_shipY > 1024:
                    print 'out of range'
                    continue

                t1 = int(line[1])
                score = int(line[4])

                dist = getDist(current_shipX, current_shipY, past_shipX, past_shipY)
                if dist > 0:
                    out_file.write(str(dist) + '\n')
                    path_lengths.append(dist)
                    zoom_list.append( [current_shipX, current_shipY] )


                    past_shipX2 = past_shipX
                    past_shipY2 = past_shipY
                    past_shipX = current_shipX
                    past_shipY = current_shipY
                    fixations.append((past_shipX, past_shipY))

                elif dist == 0:
                    #print '0 dist'
                    pass
                else:
                    print 'DISTANCE LESS THAN 0\n'
                    print dist
                    print '{0} {1} {2} {3}'.format(current_shipX, current_shipY, past_shipX, past_shipY)
                
            
            
                
            if line[0] == 'Gathered':
                collections.append((current_shipX, current_shipY))
            
            if line[0] == 'FINISHED':
                totalScore = score
    
    pieces_segs = []
    pieces_temp = []
    
    ends_segs = []
    ends_temp = None
    
    # perform some quick summary stats
    for i in range(len(zoom_list)):
        if i > 2:
            angle = getAngle(
                        zoom_list[i-2][0],zoom_list[i-2][1], zoom_list[i][0],zoom_list[i][1], zoom_list[i-1][0],zoom_list[i-1][1]
                        )
            if angle != -1:
                angles.append(angle)
                angle_file.write(str(angle) + '\n')
            
            # sum pieces
            pieces_temp.append(getDist(zoom_list[i-2][0],zoom_list[i-2][1], zoom_list[i-1][0],zoom_list[i-1][1]))
            if angle > angle_thresh:
                pieces_segs.append(np.sum(pieces_temp))
                pieces_temp = []
            
            # ends only segs
            if ends_temp is None:
                ends_temp = (zoom_list[i-2][0], zoom_list[i-2][1])
            if angle > angle_thresh:
                dist = getDist(ends_temp[0],ends_temp[1], zoom_list[i-1][0],zoom_list[i-1][1])
                ends_segs.append(dist)
                ends_temp = None
                thresh_angles.append(angle)
                
                
    np.savetxt('datas\\{0}-{1}-{2}-segs.txt'.format(out_name, name, trial_num), pieces_segs, fmt='%5.10f')
    np.savetxt('datas\\{0}-{1}-{2}-segs_e.txt'.format(out_name, name, trial_num), ends_segs, fmt='%5.10f')
    np.savetxt('datas\\{0}-{1}-{2}-thresh_angles.txt'.format(out_name, name, trial_num), thresh_angles, fmt='%5.10f')
    
    #get the mean path length
    mean = mean_func(path_lengths)
    
    #get the standard deviation
    dev = std(path_lengths)
    
    
    
    
    #print angles
    angle_mean = mean_func(angles)
    angle_dev = std(angles)
    
    seg_mean = np.mean(pieces_segs)
    seg_dev = np.std(pieces_segs)
    
    effic = (totalScore / float(num_stars)) / np.sum(path_lengths)
    assert not np.isnan(effic), '{} {} {}'.format(totalScore, float(num_stars), np.sum(path_lengths))
                                                  
    seg_effic = (totalScore / float(num_stars)) / np.sum(pieces_segs)
    assert not np.isnan(seg_effic), '{} {} {}'.format(totalScore, float(num_stars), np.sum(pieces_segs))
    
    
    #print out the summary info
    outfile.write( '{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}\t{8}\t{9}\t{10}\t{11}\t{12}\n'.format( 
                subj_id, trial_num, num_stars, clustering, totalScore, mean, dev,
                angle_mean, angle_dev, seg_mean, seg_dev, effic, seg_effic)
                )
    

    out_file.close()

In [ ]:
#script that acts as the driver for forageStats
#parses the data into trials, and sends it to forageStats

import os, re, sys

#os.chdir(sys.argv[1])
#os.chdir('c:/compile/foraging/simpleTurk/sorted/')
#os.chdir('c:/compile/foraging/blankSimple/sorted/')
roots = ['C:\\Compile\\foraging\\guidedForage\\sorted']

def file_check(file_name):
    if (re.findall('Log\d\d?-\d\d\d?-.{5}\.txt', file_name) or 
        re.findall('Log\d\d?-\d\d\d?-\d{5}\.txt', file_name) or 
        re.findall('Log\d\d?-\d\d\d?-.{5}-\d+-\d+\.\d+\.txt', file_name) or 
        re.findall('Log\d\d?-\d\d\d?-.{5}-\d+-\d+\.txt', file_name) or
        re.findall('Log\d-\d\d\d\d-\d\d\d\d\d\d?\d?(-0-75)?.txt', file_name) or
        re.findall('Log\d-\d\d\d\d-.{7}\.txt', file_name)
        ):
        return True

for root in roots:
    os.chdir(root)
    dirs = os.listdir('.')
    dirs = filter( lambda x: os.path.isdir(x), dirs )
    
    for dir in dirs:
        os.chdir(dir)
        files = os.listdir('.')
        files = filter( file_check, files )
        
        for file in files:
            with open(file) as f:
                print file
                string = ''
                name = ''
                out_name = ''
                recording = 0
                for line in f:
                    if line[:11] == "Start Trial":
                        if line[12].isdigit() and (int(line[12]) == 1):
                            recording = 1
                            id = line[-8:].strip()
                            trial_num = line[12]
                            name = '{0}'.format(id, trial_num)
                            string = ''
                    if recording:
                        string += line
                        if line == '\n':
                            recording = 0
                            out_name = 'uno'
                            get_stats(name, out_name, trial_num, string)

                get_stats(name, 'uno', trial_num, string)
    
        os.chdir('..')


In [48]:
cd ..

C:\Compile\foraging\guidedForage


# Read Data

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import matplotlib
import scipy, scipy.stats
import os, csv, re
import lnbin #note this a local file
import aicnew #also local
import powerlaw
%matplotlib inline
os.chdir('sorted/')

In [16]:
#Load in the data into one monster data frame
df = DataFrame()
for dr in [dirs for dirs in os.listdir('.') if os.path.isdir(dirs)]:
    temp = pd.read_table(os.path.join(dr, 'datas/data-uno.txt'), 
                         names=['subj_id', 'trial_num', 'num_stars', 'clustering', 'totalScore', 'mean', 'dev', 'angle_mean', 'angle_dev', 'seg_mean', 'seg_dev', 'efficiency', 'seg_efficiency']
                         )
    df = pd.concat([df, temp], ignore_index=True)
    
df = df[['subj_id', 'trial_num', 'num_stars', 'clustering', 'totalScore', 'mean', 'dev', 'angle_mean', 'angle_dev', 'seg_mean', 'seg_dev', 'efficiency', 'seg_efficiency']]

In [18]:
df

,subj_id,trial_num,num_stars,clustering,totalScore,mean,dev,angle_mean,angle_dev,seg_mean,seg_dev,efficiency,seg_efficiency
0,205189,1,100,1,3,167.871942,140.890006,27.537172,32.253910,265.598856,240.548013,0.000001,0.000001
1,503985,1,100,1,49,50.292102,55.291494,44.989178,47.838394,64.458229,76.021580,0.000032,0.000034
2,608015,1,100,1,0,94.015794,107.279217,25.339579,32.677060,159.470498,180.735021,0.000000,0.000000
3,872628,1,100,1,56,100.115920,128.980435,31.905346,38.424651,157.646417,227.228972,0.000019,0.000019
4,912291,1,100,1,54,67.792113,79.793727,19.313885,25.064542,140.118519,214.564253,0.000027,0.000027
5,945447,1,100,1,51,100.150689,89.258306,26.001462,24.763394,137.548777,160.736264,0.000017,0.000018
6,960490,1,100,1,60,80.761246,117.174764,37.846384,40.497763,109.815242,158.966947,0.000025,0.000026
7,1171912,1,100,1,47,86.249812,97.478555,19.962597,27.413011,156.782247,179.936629,0.000018,0.000018
8,1210557,1,100,1,58,71.267626,110.687865,25.964859,33.907146,115.611952,186.858150,0.000027,0.000028
9,1401884,1,100,1,28,70.805733,59.049981,19.296060,23.982178,132.620359,135.992593,0.000013,0.000014


### Hi
#### Mark the trials that perform above and below the median for each condiion

In [17]:
#puts each "condition" in a list that can be easily iterated through as needed
num_stars = [100, 600, 1100]
clustering = [1, 3, 5]

datas = []
for n_stars in num_stars:
    for cluster in clustering:
        datas.append(df[ (df['num_stars'] == n_stars) & (df['clustering'] == cluster) ]) 

In [18]:
hi = pd.Series()
for data in datas:
    med = data['totalScore'].median()
    temp = data['totalScore'] > med
    hi = pd.concat([hi, temp], ignore_index=True)
df['hi'] = hi

## Raw lnbin
### Generate lograthmically binned histograms of the raw data

In [20]:
num_paths = []
dat = {'midpts':[], 'freq':[], 'id':[], 'clicks_orig_num_paths':[], 'clicks_pruned_num_paths':[]}
folders = [dirs for dirs in os.listdir('.') if os.path.isdir(dirs)]
for folder in folders:
    print folder
    files = [files for files in os.listdir(os.path.join(folder, 'datas')) if files[-10:] == 'clicks.txt']
    for filex in files:
        paths = []
        dat['id'].append(int(filex[4:11]))
        trial_num = int(filex[-12])
        with open(os.path.join(folder, 'datas', filex)) as f:
            for line in f:
                paths.append(float(line.split()[0]))
#XMIN IS HERE
        xmin = 0
        paths = np.array(paths)
        num_paths.append( paths.shape[0] )
        (midpts, freq) = lnbin.lnbin(paths[paths >= float(xmin)], 10)
        dat['midpts'].append(midpts)
        dat['freq'].append(freq)
        dat['clicks_orig_num_paths'].append(paths.size)
        dat['clicks_pruned_num_paths'].append(paths[paths >= float(xmin)].size)
        
segs = DataFrame(dat)
segs = segs[ ['midpts', 'freq', 'clicks_orig_num_paths', 'clicks_pruned_num_paths'] ]

df = pd.merge(df, segs, left_index=True, right_index=True)

0100stars1density
0100stars3density
0100stars5density
0600stars1density
0600stars3density
0600stars5density
1100stars1density
1100stars3density
1100stars5density


### Seg lnbin
#### Perform lograthimc binning of the data segmented into "Levy Flights" as in Kerster et al., 2016

In [21]:
num_paths = []
dat = {'seg_midpts':[], 'seg_freq':[], 'id':[], 'orig_num_paths':[], 'pruned_num_paths':[]}
folders = [dirs for dirs in os.listdir('.') if os.path.isdir(dirs)]
for folder in folders:
    print folder
    files = [files for files in os.listdir(os.path.join(folder, 'datas')) if files[-8:] == 'segs.txt']
    for filex in files:
        paths = []
        dat['id'].append(int(filex[4:11]))
        trial_num = int(filex[-10])
        with open(os.path.join(folder, 'datas', filex)) as f:
            for line in f:
                paths.append(float(line.split()[0]))
#XMIN IS HERE
        xmin = 0
        paths = np.array(paths)
        num_paths.append( paths.shape[0] )
        (midpts, freq) = lnbin.lnbin(paths[paths >= float(xmin)], 10)
        dat['seg_midpts'].append(midpts)
        dat['seg_freq'].append(freq)
        dat['orig_num_paths'].append(paths.size)
        dat['pruned_num_paths'].append(paths[paths >= float(xmin)].size)
        
segs = DataFrame(dat)
segs = segs[ ['seg_midpts', 'seg_freq', 'orig_num_paths', 'pruned_num_paths'] ]

df = pd.merge(df, segs, left_index=True, right_index=True)

0100stars1density
0100stars3density
0100stars5density
0600stars1density
0600stars3density
0600stars5density
1100stars1density
1100stars3density
1100stars5density


In [46]:
#note this now uses all bins instead of just 5
def poly_seg(data):
    #first trim out NaN (in this case 0.0)
    droplist = []
    for i in range(len(data['seg_freq'])):
        if data['seg_freq'][i] == 0.0:
            droplist.append(i)
    
    x = np.log(np.delete(np.array(data['seg_midpts']), droplist))
    y = np.log(np.delete(np.array(data['seg_freq']), droplist))
    
    slope, intercept = np.polyfit(x, y, 1)
    return slope

In [47]:
slopes = df.apply(poly_seg, axis=1)
df['seg_slope'] = slopes

### Clauset (adjust xmin)
#### Use the clauset method to fit powerlaws to the distributions

In [22]:
dat = {'claus_mu':[], 'id':[], 'xmin':[]}
folders = [dirs for dirs in os.listdir('.') if os.path.isdir(dirs)]
for folder in folders:
    print folder
    files = [files for files in os.listdir(os.path.join(folder, 'datas')) if files[-8:] == 'segs.txt']
    for filex in files:
        paths = []
        dat['id'].append(int(filex[4:11]))
        trial_num = int(filex[-10])
        with open(os.path.join(folder, 'datas', filex)) as f:
            for line in f:
                paths.append(float(line.split()[0]))
        xmin = 42
        paths = np.array(paths)
        
        results = powerlaw.Fit(paths, xmin=xmin)
        #results = powerlaw.Fit(paths)
        dat['claus_mu'].append( results.power_law.alpha )
        dat['xmin'].append( results.power_law.xmin )
        
claus = DataFrame(dat)

0100stars1density
0100stars3density
0100stars5density
0600stars1density
0600stars3density
0600stars5density
1100stars1density
1100stars3density
1100stars5density


In [23]:
df['claus_mu'] = claus['claus_mu']

In [22]:
df.head()

,subj_id,trial_num,num_stars,clustering,totalScore,mean,dev,angle_mean,angle_dev,seg_mean,seg_dev,efficiency,seg_efficiency,aic,claus_mu
0,205189,1,100,1,3,167.871942,140.890006,27.537172,32.253910,265.598856,240.548013,0.000001,0.000001,lognormal,1.639167
1,503985,1,100,1,49,50.292102,55.291494,44.989178,47.838394,64.458229,76.021580,0.000032,0.000034,lognormal,2.362616
2,608015,1,100,1,0,94.015794,107.279217,25.339579,32.677060,159.470498,180.735021,0.000000,0.000000,lognormal,1.799232
3,872628,1,100,1,56,100.115920,128.980435,31.905346,38.424651,157.646417,227.228972,0.000019,0.000019,lognormal,1.693319
4,912291,1,100,1,54,67.792113,79.793727,19.313885,25.064542,140.118519,214.564253,0.000027,0.000027,lognormal,1.917598


### AIC
#### Use AIC to determine the model which best fits the distributions

In [24]:
dic = {'folder':[], 'aic_raw':[]}
for base in ['c:\\compile\\foraging\\guidedForage\\sorted']:
    folders = [dirs for dirs in os.listdir('.') if os.path.isdir(dirs)]
    for folder in folders:
        print folder
        files = [files for files in os.listdir(os.path.join(base, folder, 'datas')) if files[-8:] == 'segs.txt']      
        for item in files:
            data = np.loadtxt(os.path.join(base, folder, 'datas', item))
            

            dic['folder'].append(folder)
            _, max_w, __ = aicnew.aic(data)
            dic['aic_raw'].append(max_w)
            # _, max_w, __ = aicnew.aic(data[data >= 42])
            # dic['aic_trim'].append(max_w)

aic_f = pd.DataFrame(dic)

0100stars1density
0100stars3density
0100stars5density
0600stars1density
0600stars3density
0600stars5density
1100stars1density
1100stars3density
1100stars5density


In [25]:
aic_f['aic_raw'] = aic_f['aic_raw'].astype('category', categories=['normal', 'lognormal', 'exponential', 'pareto', 'boundedpl'], ordered=True)

In [26]:
df['aic'] = aic_f['aic_raw']

In [27]:
def aic_cat(dist):
    if dist == 'normal':
        return 1
    elif dist == 'lognormal':
        return 2
    elif dist == 'exponential':
        return 3
    elif dist == 'pareto':
        return 4
    elif dist == 'boundedpl':
        return 5

In [28]:
aic_f['aic_raw'] = aic_f['aic_raw'].apply(aic_cat)

In [12]:
df

,subj_id,trial_num,num_stars,clustering,totalScore,mean,dev,angle_mean,angle_dev,seg_mean,...,efficiency,seg_efficiency,hi,seg_midpts,seg_freq,orig_num_paths,pruned_num_paths,claus_mu,aic,aic_clicks
0,205189,1,100,1,3,167.871942,140.890006,27.537172,32.253910,265.598856,...,0.000001,0.000001,False,"[3.69456906791, 6.82499607234, 12.6078496656, ...","[0.0, 0.0, 0.000676753710402, 0.00219807891334...",188,188,1.639167,lognormal,lognormal
1,224476,1,100,1,56,56.773523,119.893350,30.713535,32.811058,74.596116,...,0.000033,0.000034,True,"[3.65441037819, 6.60485036462, 11.9373698694, ...","[0.00207115865295, 0.00229191071966, 0.0057064...",220,220,2.547678,lognormal,lognormal
2,503985,1,100,1,49,50.292102,55.291494,44.989178,47.838394,64.458229,...,0.000032,0.000034,True,"[3.56499391922, 6.13179202767, 10.5466865646, ...","[0.0, 0.00265232021444, 0.0131073910932, 0.019...",224,224,2.362616,lognormal,lognormal
3,539569,1,100,1,59,85.646711,99.711900,26.724675,31.376190,133.289195,...,0.000023,0.000024,True,"[3.62727652773, 6.45881779081, 11.5007297999, ...","[0.0, 0.00140777442865, 0.00711546381412, 0.01...",188,188,1.811242,boundedpl,lognormal
4,608015,1,100,1,0,94.015794,107.279217,25.339579,32.677060,159.470498,...,0.000000,0.000000,False,"[3.64666565387, 6.56294668432, 11.8114116482, ...","[0.0, 0.00145230052557, 0.00322785157366, 0.00...",176,176,1.799232,lognormal,lognormal
5,872628,1,100,1,56,100.115920,128.980435,31.905346,38.424651,157.646417,...,0.000019,0.000019,True,"[3.71727539977, 6.95160759564, 13.0000720869, ...","[0.00228546146666, 0.00244423741422, 0.0130702...",185,185,1.693319,lognormal,lognormal
6,912291,1,100,1,54,67.792113,79.793727,19.313885,25.064542,140.118519,...,0.000027,0.000027,True,"[3.7466008206, 7.11743169394, 13.5210118034, 2...","[0.00285936726934, 0.00150516481483, 0.0110924...",143,143,1.917598,lognormal,lognormal
7,945447,1,100,1,51,100.150689,89.258306,26.001462,24.763394,137.548777,...,0.000017,0.000018,True,"[3.68354241773, 6.76406951398, 12.4208251735, ...","[0.0, 0.0, 0.00247308431234, 0.0063972031932, ...",211,211,1.948320,lognormal,lognormal
8,960490,1,100,1,60,80.761246,117.174764,37.846384,40.497763,109.815242,...,0.000025,0.000026,True,"[3.64610032273, 6.55989485832, 11.8022590558, ...","[0.00215110413136, 0.00358685998952, 0.0152845...",214,214,1.890562,lognormal,lognormal
9,1171912,1,100,1,47,86.249812,97.478555,19.962597,27.413011,156.782247,...,0.000018,0.000018,True,"[3.64496463916, 6.55376695958, 11.7838897253, ...","[0.0, 0.00157289361712, 0.0122469997962, 0.005...",163,163,1.806409,lognormal,lognormal


In [27]:
# Do AIC without angle threshold
dic = {'folder':[], 'aic_raw':[]}
for base in ['c:\\compile\\foraging\\guidedForage\\sorted']:
    folders = [dirs for dirs in os.listdir('.') if os.path.isdir(dirs)]
    for folder in folders:
        print folder
        files = [files for files in os.listdir(os.path.join(base, folder, 'datas')) if files[-10:] == 'clicks.txt']      
        for item in files:
            data = np.loadtxt(os.path.join(base, folder, 'datas', item))
            

            dic['folder'].append(folder)
            _, max_w, __ = aicnew.aic(data)
            dic['aic_raw'].append(max_w)

aic_f = pd.DataFrame(dic)

0100stars1density
0100stars3density
0100stars5density
0600stars1density
0600stars3density
0600stars5density
1100stars1density
1100stars3density
1100stars5density


In [28]:
aic_f['aic_raw'] = aic_f['aic_raw'].astype('category', categories=['normal', 'lognormal', 'exponential', 'pareto', 'boundedpl'], ordered=True)

In [9]:
aic_f['aic_raw'].describe()

count          1033
unique            3
top       lognormal
freq           1007
Name: aic_raw, dtype: object

In [29]:
df['aic_clicks'] = aic_f['aic_raw']

In [13]:
df.to_csv('df.csv')
df.to_pickle('df.pik')

In [6]:
df = pd.read_pickle('df.pik')

# Get AF
## Perform Allen Factor analyses as a second measure of "clustering" in the movements

In [6]:
import os
cwd = os.getcwd()
os.chdir('C:\\Users\\Bryan\\Documents\\GitHub\\utilities\\cy_af2d')
import af2d
os.chdir(cwd)
os.chdir('sorted')

In [7]:
os.getcwd()

'C:\\Compile\\foraging\\guidedForage\\sorted'

In [23]:
def af_convert(points):
    space = np.zeros((1280, 1024)).astype(np.int32)
    for pnt in points:
    
        try:
            space[int(pnt[0])-1, int(pnt[1])-1] += 1
        except:
            import pdb; pdb.set_trace()
    return space.astype(int)

In [46]:
import numpy as np

def get_af( name, out_name, trial_num, file):
    #check if the directory already exists, if it doesn't make it
    if not os.path.exists('datas'):
        os.mkdir('datas')
    #declare a few file objects
    #out_file = open('datas\\{0}-{1}-{2}-clicks.txt'.format(out_name, name, trial_num), 'w') #zoomed out

    #segs_file = open('datas\\{0}-{1}-{2}-segs.txt'.format(out_name, name, trial_num), 'w')
    #angle_file = open('datas\\{0}-{1}-{2}-angle.txt'.format(out_name, name, trial_num), 'w') #angles!
    #angle_thresh_file = open('datas\\{0}-{1}-{2}-anglethresh.txt'.format(out_name, name, trial_num), 'w')
    outfile = open('datas\\data-{0}.txt'.format(out_name), 'a') #summary stats
    
    #variables
    space = [ [0 for col in range(1280)] for row in range(1024) ]
    subj_id = name[0:7]
    trial_id = 0
    clustering = 0 #0 is random, 1 is clustered [-8]
    num_stars = 0
    score = 0
    totalScore = 0
    #path_mean_length
    path_lengths = []
    in_lengths = []
    angles = []
    thresh_angles = []
    zoom_lengths = []
    zoom_list = []
    skip_lengths = []
    #path_dev_length
    number_of_dives = 0
    
    #start the ship in the center
    past_shipX2 = 640.0 
    past_shipY2 = 512.0
    past_shipX = 640.0 
    past_shipY = 512.0

    current_shipX = 640.0
    current_shipY = 512.0
    fixations = []
    collections = []
    outX = 0.0
    outY = 0.0
    zX = 0.0
    zY = 0.0
    zX2 = 640.0
    zY2 = 512.0
    zoomed = 0

    
    angle_thresh = 10
    angle_segments = [] #temp list for adding segments together
    angle_paths = [] #hold the full pahs determined by angle changes
    
    landed = 1 #variable to keep track of whether the probe has landed
    file = file.split('\n')
    file_length = len(file)
    for iter, line in enumerate(file):
        if len(line) > 0:
            #print line
           
            if line[:5] == "maps/":
                num_stars = int(line[5:9])
                clustering = int(line[14])
                
                
            line = line.split()
            #if an interesting event occurs...
            if line[0] == 'Click':

                current_shipX = float(line[2])
                current_shipY = float(line[3])
                if current_shipX <= 0 or current_shipY <= 0 or current_shipX > 1280 or current_shipY > 1024:
                    print 'out of range'
                    continue

                t1 = int(line[1])
                score = int(line[4])

                dist = getDist(current_shipX, current_shipY, past_shipX, past_shipY)
                if dist > 0:
                    #out_file.write(str(dist) + '\n')
                    path_lengths.append(dist)
                    zoom_list.append( [current_shipX, current_shipY] )


                    past_shipX2 = past_shipX
                    past_shipY2 = past_shipY
                    past_shipX = current_shipX
                    past_shipY = current_shipY
                    fixations.append((past_shipX, past_shipY))

                elif dist == 0:
                    #print '0 dist'
                    pass
                else:
                    print 'DISTANCE LESS THAN 0\n'
                    print dist
                    print '{0} {1} {2} {3}'.format(current_shipX, current_shipY, past_shipX, past_shipY)
                
            
            
                
            if line[0] == 'Gathered':
                collections.append((current_shipX, current_shipY))
            
            if line[0] == 'FINISHED':
                totalScore = score
    
    pieces_segs = []
    pieces_temp = []
    
    ends_segs = []
    ends_temp = None
    
    af, series = af2d.af2d(af_convert(zoom_list))
    
    outfile.write( '{}\t{}\t{}\t{}\t[ {} ]\t[ {} ]\n'.format(
        subj_id, trial_num, num_stars, clustering, ' '.join(af.astype(str)), ' '.join(series.astype(str))) )
    

    outfile.close()
    return af

In [ ]:
#script that acts as the driver for forageStats
#parses the data into trials, and sends it to forageStats

import os, re, sys

#os.chdir(sys.argv[1])
#os.chdir('c:/compile/foraging/simpleTurk/sorted/')
#os.chdir('c:/compile/foraging/blankSimple/sorted/')
roots = ['C:\\Compile\\foraging\\guidedForage\\sorted']

def file_check(file_name):
    if (re.findall('Log\d\d?-\d\d\d?-.{5}\.txt', file_name) or 
        re.findall('Log\d\d?-\d\d\d?-\d{5}\.txt', file_name) or 
        re.findall('Log\d\d?-\d\d\d?-.{5}-\d+-\d+\.\d+\.txt', file_name) or 
        re.findall('Log\d\d?-\d\d\d?-.{5}-\d+-\d+\.txt', file_name) or
        re.findall('Log\d-\d\d\d\d-\d\d\d\d\d\d?\d?(-0-75)?.txt', file_name) or
        re.findall('Log\d-\d\d\d\d-.{7}\.txt', file_name)
        ):
        return True

for root in roots:
    os.chdir(root)
    dirs = os.listdir('.')
    dirs = filter( lambda x: os.path.isdir(x), dirs )
    
    for dir in dirs:
        os.chdir(dir)
        files = os.listdir('.')
        files = filter( file_check, files )
        
        for file in files:
            f = open(file)
            print file
            string = ''
            name = ''
            out_name = ''
            recording = 0
            for line in f:
                if line[:11] == "Start Trial":
                    if line[12].isdigit() and (int(line[12]) == 1):
                        recording = 1
                        id = line[-8:].strip()
                        trial_num = line[12]
                        #print trial_num
                        name = '{0}'.format(id, trial_num)
                        string = ''
                if recording:
                    string += line
                    # if line[:4] == "maps":
                        # if line[7] == "s":
                        
                        
                            # out_name = 'random'
                        # else:
                            # out_name = 'clustered'
                    if line == '\n':
                        recording = 0
                        out_name = 'uno'
                        print 'beep'
                        get_stats(name, out_name, trial_num, string)
            
            al = get_af(name, 'af', trial_num, string)
    
            f.close()
        os.chdir('..')


###### pull AF out of text files and add to df

In [2]:
#Load in the data into one data frame
adf = DataFrame()
for dr in [dirs for dirs in os.listdir('.') if os.path.isdir(dirs)]:
    temp = pd.read_table(os.path.join(dr, 'datas/data-af.txt'), 
                         names=['subj_id', 'trial_num', 'num_stars', 'clustering', 'af', 'boxSeries' ]
                         )
    adf = pd.concat([adf, temp], ignore_index=True)
    
adf = adf[['subj_id', 'trial_num', 'num_stars', 'clustering', 'af', 'boxSeries']]

In [3]:
def str_to_list(string):
    lst = np.array([float(item) for item in string.replace('[','').replace(']','').split()])
    return lst

In [4]:
adf['af'] = adf['af'].apply(str_to_list)
adf['boxSeries'] = adf['boxSeries'].apply(str_to_list)

In [9]:
df['af'] = adf['af']
df['boxSeries'] = adf['boxSeries']

In [12]:
# sanity check a row
df['af'].ix[2]

array([  1.00176014,   1.00185241,   1.00371176,   0.97533469,
         0.6395285 ,   0.78081059,   2.62549708,   7.73439153,  57.66527778])